<center><img src='../../img/ai4eo_logos.jpg' alt='Logos AI4EO MOOC' width='80%'></img></center>

<hr>

<a href="https://www.futurelearn.com/courses/artificial-intelligence-for-earth-monitoring/1/steps/1280523"><< Back to FutureLearn</a><br>

# Training of a Neural Network for estimating total column ozone from EO data

<i>by Davide De Santis, University of Tor Vergata, Rome, Italy</i>

<hr>

## Watch the video tutorial

In [ ]:
from IPython.display import HTML
HTML('<div align="center"><iframe src="https://player.vimeo.com/video/631907302?h=3debddcc22" width="640" height="360" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen align="middle"></iframe></div>')     

<br>

<hr>

## Introduction

This workflow guides you through the process of training a `neural network` based on [Sentinel-5P/TROPOMI](http://www.tropomi.eu/) data in order to predict `total column ozone concentration`. The labelled data base on a Sentinel-5P/TROPOMI scene which was acquired over Sicily (Italy) on December 29, 2018.

The labelled data takes information from the following Sentinel-5P/TROPOMI data:
* [Sentinel-5P/TROPOMI Level 2 - Product User Manual - Ozone Total Column](https://sentinels.copernicus.eu/documents/247904/2474726/Sentinel-5P-Level-2-Product-User-Manual-Ozone-Total-Column.pdf/6d813a34-2811-4a29-9f79-b6a0dda95901?t=1611302543710)
* [Sentinel-5P/TROPOMI Level 1B -  Product Readme File](https://sentinels.copernicus.eu/documents/247904/3541451/Sentinel-5P-Level-1b-Product-Readme-File.pdf/a89d82ce-7414-43e6-ac77-0c371ed1b096?t=1572967657000)

## Machine-Learning Algorithm

Neural networks have shown promising results for atmospheric inversion problems, in particular for ozone column abundance estimation. For this reason, the workflow trains a [sequential neural network with Keras](https://keras.io/api/models/sequential/), which is a linear stack of layers. There are two ways to build Keras models:
* in a `sequential` way and
* a `functional` way.

The `sequential` API allows you to create models in a sequential way, which means you add one layer after the other. A sequential model is applicable for most problems. 

The network is designed with 23 input nodes (spectral radiances and zenith angles), a single hidden layer consinsting of 20 neurons and 1 output node (ozone total column concentration) both with a [sigmoidal activation function](https://keras.io/api/layers/activations/) associated.





## Data

This workflow makes use of the .csv file [AI4EO_Ozone_Estimation_Sentinel-5P](./AI4EO_Ozone_Estimation_Sentinel-5p.csv), which contains labelled data from a Sentinel-5P/TROPOMI scene acquired over Sicily on December 29, 2018.

The file contains 2500 row entries and 26 columns. The 2500 rows resemble 2500 point locations across Sicily, Italy. The 26 columns represent the following information:
* Column 1-21: `Spectral radiance value for 21 wavelengths [rad_325.0_nm, rad_335.0_nm]` sensitive to the ozone column abundance and derived from the TROPOMI Level-1B Band-3 product
* Column 22: `solar zenith angle [sza]` derived from the TROPOMI Level-1B Band-3 product
* Column 23: `sensor zenith angle [vza]` derived from the TROPOMI Level-1B Band-3 product
* Column 24: `latitude [lat]`
* Column 25: `longitude [lon]`
* Column 26: `Total Ozone column abundance [ozone_total_column_[DU]]` derived from the TROPOMI Level-2 Total Column Ozone product

## Further resources

* [Application of neural algorithms for a real-time estimation of ozone profiles from GOME measurements](https://ieeexplore.ieee.org/document/1105913)
* [Tropospheric ozone column retrieval at northern mid-latitudes from the Ozone Monitoring Instrument by means of a neural network algorithm](https://amt.copernicus.org/articles/4/2375/2011/)
* [Tropospheric Ozone Column Retrieval From ESA-Envisat SCIAMACHY Nadir UV/VIS Radiance Measurements by Means of a Neural Network Algorithm, in IEEE Transactions on Geoscience and Remote Sensing](https://ieeexplore.ieee.org/iel5/36/4358825/06008635.pdf?casa_token=lqc9umtUPLEAAAAA:IjRD1HqEc0H3BeSNdFSr-AVWliUSUUTfxTLGdT9fGRJKADbxf-sIS5uDn-ISDyFPXnJtA7m3Ug)

<hr>

## Notebook outline


* [1 - Prepare input and output data for model training](#input_output_ozone)
* [2 - Define and compile a sequential neural network with Keras](#model_definition_ozone)
* [3 - Training (fitting) of the sequential neural network](#fitting_ozone)
* [4 - Use the trained sequential neural network to predict total column ozone estimates](#predict_ozone)
* [5 - Evaluate the accuracy of the model predictions](#evaluate_ozone)

<hr>

### Import libraries

In [ ]:
## BEGIN S3FS IMPORT SNIPPET ##
import os, sys
s3_home =  os.getcwd()
try: sys.path.remove(s3_home) # REMOVE THE S3 ROOT FROM THE $PATH
except Exception: pass
current_dir = os.getcwd()
os.chdir('/home/jovyan') # TEMPORARILY MOVE TO ANOTHER DIRECTORY

# BEGIN IMPORTS #
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
# END IMPORTS #

os.chdir(current_dir) # GO BACK TO YOUR PREVIOUS DIRECTORY
sys.path.append(s3_home) # RESTORE THE S3 ROOT IN THE $PATH
## END S3FS IMPORT SNIPPET ##

<hr>

## <a id='input_output_ozone'></a>1. Prepare input and output data for model training

### Load the training dataset

The first step is to load the training dataset [AI4EO_Ozone_Estimation_Sentinel-5p](./AI4EO_Ozone_Estimation_Sentinel-5p.csv), which is provided as a `.csv` file. The file contains 2500 row entries and 26 columns. The 2500 rows resemble 2500 point locations across Sicily, Italy. The 26 columns represent the following information:
* Column 1-21: `Spectral radiance value for 21 wavelengths [rad_325.0_nm, rad_335.0_nm]` sensitive to the ozone column abundance and derived from the TROPOMI Level-1B Band-3 product
* Column 22: `solar zenith angle [sza]` derived from the TROPOMI Level-1B Band-3 product
* Column 23: `sensor zenith angle [vza]` derived from the TROPOMI Level-1B Band-3 product
* Column 24: `latitude [lat]`
* Column 25: `longitude [lon]`
* Column 26: `Total Ozone column abundance [ozone_total_column_[DU]]` derived from the TROPOMI Level-2 Total Column Ozone product

With the Python library [pandas](https://pandas.pydata.org/) and the function `read_csv()`, you can easily read a `.csv` file and inspect the content of the file. The function `.shape()` shows you that the loaded data file has 2500 rows and 26 columns.

In [ ]:
dataset = pd.read_csv("./AI4EO_Ozone_Estimation_Sentinel-5p.csv") 
dataset

### Define input and output variables of the neural network model

In a next step, you define the columns and rows of the loaded dataset that shall serve as input (X) and ouput (y) variables of the neural network model. The first 23 columns featuring spectral radiance values for 21 wavelengths as well as solar and sensor zenith angles shall serve as input (X) variables. The total column ozone information serves as output (y) variable, which the model shall be able to predict.

In [ ]:
X = dataset.values[:, 0:23]
y = dataset.values[:, 25:26]

X.shape, y.shape

<br>

### Split the dataset in training, validation and testing subsets

Now, you can split the input and output variables into subsets for `training`, `validation` and `testing`:
* `training data`: is the actual sample used to train the Machine Learning model that learns from this data
* `validation data`: used to evaluate how well the model is fitted onto the training data, while the model hyperparameters are tuned. However, the network does not learn from this data
* `test data`: used to provide an evaluation of the final model fit and these data are only used when a model is completely trained based on training and validation data

[scikit-learn](https://scikit-learn.org/stable/) offers a function called `train_test_split()` which creates four subsets based on the input and output variables, `X` and `y` respectively. 

The function takes the following kwargs:
* `arrays`: input and output data arrays
* `test_size`: a float number representing the proportion of the dataset to include in the test subset
* `random_state`: An integer assuring reproducibility of the random shuffling of the data  

First, you want to split the data into 60% training and 40% validation and test data.

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X, 
                                                                    y, 
                                                                    test_size=0.4, 
                                                                    random_state=32)

X_train.shape, X_val_and_test.shape, y_train.shape, y_val_and_test.shape

<br>

In a following step, you want split the validation and test data subset and use 55% for validation and 45% for testing the neural network. 

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, 
                                                y_val_and_test, 
                                                test_size=0.45, 
                                                random_state=32)

X_val.shape, X_test.shape, y_val.shape, y_test.shape

<br>

### Normalisation of input and output data

The next step is to normalise the input and output data. `Normalisation` is a common pre-processing step in Machine Learning to bring variables that are measured in different units to a common scale. Normalisation is not needed for all Machine Learning algorithms, but it is critical for `neural networks`.

You can use the `MinMaxScaler`class from the scikit-learn library to scale the input and output variables. You can specify the minimum and maximum bounds of the `feature_range` as keyword argument.

Let us define two `MinMaxScaler` objects, for the input and output data respectively with different feature ranges:
* `input_scaler`: [-5, 5]
* `output_scaler`: [0, 1]

The feature range is adjusted to the `sigmoidal activation function` used for each neuron in the neural network. The input and output features are scaled according to the function domain and co-domain.


In [ ]:
input_scaler = MinMaxScaler(feature_range=(-5, 5))
output_scaler = MinMaxScaler(feature_range=(0, 1))

<br>

The next step is to fit the `MinMaxScaler` object to the data space. You can fit the two `MinMaxScaler` objects above to the input (`X`) and output (`y`) data spaces.

Once the `MinMaxScaler` object are fitted, you can scale (transform) the input and output data subsets for `training`, `validation` and `testing` with the function `transform()`.

> **Note:** Since we do not use the test data subset during the training phase, it is not required to scale the variable `y_test`. We only need the data in the original scale to compare them with the output of the network in order to estimate the accuracy scores.

In [ ]:
input_scaler.fit(X)
X_train_scaled = input_scaler.transform(X_train)
X_val_scaled = input_scaler.transform(X_val)
X_test_scaled = input_scaler.transform(X_test)

# with the output scaler we fit all the output space and then scale each splitted part of the dataset used as output for the neural network
output_scaler.fit(y)
y_train_scaled = output_scaler.transform(y_train)
y_val_scaled = output_scaler.transform(y_val)

<br>

## <a id='model_definition_ozone'></a>2. Define and compile a sequential neural network with Keras

In a next step, you can define the sequential neural network with `keras.Sequential()`. The aim is to define a neural network with 23 input nodes (spectral radiances and zenith angles), a single hidden layer consisting of 20 neurons and 1 output node (total column ozone concentration). The input dimension has to match with the number of columns of the input (`X`) data.

You want to add the following `dense layers`:
* Layer 1: `dense layer` with 20 neurons, 23 input dimensions and an associated `sigmoid` activation function
* Layer 2: `dense layer` with 1 final neuron and an associated `sigmoid` activation function

>**Note:**
See an overview of possible layer activation functions [here](https://keras.io/api/layers/activations/).

In [ ]:
model_nn_for_o3 = Sequential([Dense(units=20, 
                                    activation='sigmoid', 
                                    input_shape=(23,)),
                              Dense(units=1, 
                                    activation='sigmoid')])

<br>

The last step before model fitting is to compile (configure) the model. You can do this with the `.compile()` function. Let us define the following configurations:
* `optimizer='sgd'` - use a `stochastic gradient descent` as [optimizer](https://keras.io/api/optimizers/)
* `loss='mean_squared_error'` - use mean squared error as [loss function](https://keras.io/api/losses/)
* `metrics=['mae']` - Use mean absolute error as a [metric](https://keras.io/api/metrics/) of model performance evaluation

In [ ]:
model_nn_for_o3.compile(optimizer='sgd',  
                        loss='mean_squared_error',  
                        metrics=['mae'])

<br>

## <a id='fitting_ozone'></a>3. Training (fitting) of the sequential neural network

`Model fitting` is the process that trains the sequential neural network with the defined training input and output data subsets. You can specify the following keyword arguments:
* `input (X)` and `output (y)` data: here specify the input and output training data for the model
* `validation_data`: here we enter the validation data subsets `X_val_scaled` and `y_val_scaled` and our model outputs are validated against these validation data after each epoch (training cycle)
* `epochs`: number of training cycles
* `batch_size`: defines the size of a training data subset (e.g. 10 samples) after which the weights of the network are updated
* `verbose`: set verbose=1 to print the scores after each epoch

> **Note:**
The output of the training process is a `history` object and for this reason, the output object has the name `hist`.

In [ ]:
start_training = time.time()

hist = model_nn_for_o3.fit(X_train_scaled, 
                           y_train_scaled, 
                           batch_size=10, 
                           epochs=300, 
                           validation_data=(X_val_scaled, y_val_scaled), 
                           verbose=1)

end_training = time.time()
print("Training time ", round((end_training - start_training), 3), "s" )

<br>

### Evaluate the training process

A useful evaluation of the training process is to plot the `loss function` and `mean absolute error` during the training epochs. A decreasing trend of both metrics indicates the capability of the network to learn.

Plot the `loss function` during the training epochs by plotting `loss` and `val_loss` metrics.

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

Plot the `mean absolute error` during the training epochs by plotting `mae` and `val_mae` metrics.

In [ ]:
plt.plot(hist.history['mae'])
plt.plot(hist.history['val_mae'])
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

<br>

## <a id='predict_ozone'></a>4. Use the trained sequential neural network to predict total column ozone estimates

Now you can use the trained model to predict the output variable `total_column_ozone`, based on the input data subsets `X_train_scaled`, `X_val_scaled` and `X_test_scaled`. You can use the function `model.predict()` to do so. The result is the predicted `total_column_ozone`, but still in the normalized / transformed format. For this reason, in the next step we have to inverse the transformation.

> **Note:** The estimation of the accuracy on training and validation data can be compared with that obtained on the test data. This gives us information about the variation of the error when we generalize the network performance over data that was not considered during the training.

In [ ]:
predict_train = model_nn_for_o3.predict(X_train_scaled)
predict_val = model_nn_for_o3.predict(X_val_scaled)
predict_test = model_nn_for_o3.predict(X_test_scaled)
predict_train

<br>

The `MinMaxScaler` class has a function called `inverse_transform()`, which allows you to invert the normalisation process. The function takes the normalised predicted values and returns the converted `total column ozone` estimates in [`Dobson Unit (DU)`](https://ozonewatch.gsfc.nasa.gov/facts/dobson_SH.html).

In [ ]:
predict_train_inverse = output_scaler.inverse_transform(predict_train)
predict_val_inverse = output_scaler.inverse_transform(predict_val)
predict_test_inverse = output_scaler.inverse_transform(predict_test)
predict_train_inverse

<br>

You can use the function `model.save()` to save the Keras model in the `Keras H5` format.

In [ ]:
model_nn_for_o3.save("../../../../model_nn_for_o3.h5")

<br>

## <a id='evaluate_ozone'></a>5. Evaluate the accuracy of the model predictions

As a final step, you want to evaluate the accuracy of your trained model. There are different metrics suitable for `regression problems performance evaluation` and [here](https://scikit-learn.org/stable/modules/classes.html#regression-metrics), you find a list of `regression metrics` supported by [scikit-learn](https://scikit-learn.org/stable/). For this workflow, we focus on the metrics `mean absolute error`, `mean_squared_error` and `Persons's correlation coefficient`.

Let us start with computing the `mean absolute error` between estimated and true ozone concentration values for the three data subsets *`_train`*, *`_val`* and *`_test`*. From the metrics class of the scikit-learn package, you can use the function `mean_absolute_error()` and provide the true and predicted ozone concentration values as inputs.

In [ ]:
mae_train = mean_absolute_error(y_train, predict_train_inverse)
mae_val = mean_absolute_error(y_val, predict_val_inverse)
mae_test = mean_absolute_error(y_test, predict_test_inverse)

print(f'Train MAE: {round(mae_train, 3)} DU, Val MAE: {round(mae_val, 3)} DU, Test MAE: {round(mae_test, 3)} DU')

<br>

Next, let us compute the `root mean squared error` between estimated and true ozone concentration values, again for the three data subsets *`_train`*, *`_val`* and *`_test`*. The metrics class of the scikit-learn package offers the function `mean_square_error()`.

In [ ]:
rmse_train = mean_squared_error(y_train, predict_train_inverse, squared=False)
rmse_val = mean_squared_error(y_val, predict_val_inverse, squared=False)
rmse_test = mean_squared_error(y_test, predict_test_inverse, squared=False)

print(f'Train RMSE: {round(rmse_train, 3)} DU, Val RMSE: {round(rmse_val, 3)} DU, Test RMSE: {round(rmse_test, 3)} DU')

<br>

Next, let us compute `Pearson's correlation coefficient` between the estimated and true ozone concentration values, again for the three data subsets *`_train`*, *`_val`* and *`_test`*. scikit-learn offers the function `r2_score()`, which calculates the coefficient of determination (R<sup>2</sup>), which is the square of Pearson's correlation coefficient.

In [ ]:
pearson_train = np.sqrt(r2_score(y_train, predict_train_inverse))
pearson_val = np.sqrt(r2_score(y_val, predict_val_inverse))
pearson_test = np.sqrt(r2_score(y_test, predict_test_inverse))

print(f'Train Pearson: {round(pearson_train, 3)}, Val Pearson: {round(pearson_val, 3)}, Test Pearson: {round(pearson_test, 3)}')

<br>

As a final evaluation performance, let us compare the `mean` and `standard deviation (std)` of the estimated vs real total column ozone values.

In [ ]:
print("\n" "Evaluation of mean and standard deviation of the estimated values of ozone total column compared to the actual ones")
print(f'Ozone Total Column true mean: {round(np.mean(y_test), 3)} DU --- Ozone Total Column estimated mean: {round(float(np.mean(predict_test_inverse)), 3)} DU')
print(f'Ozone Total Column true std: {round(np.std(y_test), 3)} DU --- Ozone Total Column estimated std: {round(float(np.std(predict_test_inverse)), 3)} DU')

<br>


**Conclusion:**<br>
To evaluate the performance, we can compare the mean absolute error (MAE) and mean on the test data subset. The `MAE` for the test data is much higher. Another possibility is to compare the root mean squared error (RMSE) with the standard deviation (which represents the "average" estimator). You can see that in the present workflow we could obtain a significant reduction of the RMSE with respect to the standard deviation.

<br>

<a href="https://www.futurelearn.com/courses/artificial-intelligence-for-earth-monitoring/1/steps/1174962"><< Back to FutureLearn</a><br>

<hr>

<img src='../../img/copernicus_logo.png' alt='Copernicus logo' align='left' width='20%'></img>

Course developed for [EUMETSAT](https://www.eumetsat.int/), [ECMWF](https://www.ecmwf.int/) and [Mercator Ocean International](https://www.mercator-ocean.fr/en/) in support of the [EU’s Copernicus Programme](https://www.copernicus.eu/en) and the [WEkEO platform](https://wekeo.eu/).


<br>